In [3]:
#Funtion occupancyCellChecker gets a path and finds the unique cell IDs in the occupancy grid that the path goes through.
#Assumptions
# An occupancy function exists that is called as follows ID = occupancy_get_id(x,y,z)
# Path starts at zero
# Solution has coefficient vectors of the same length
# The grid is [0, get_grid_extent] x [0, get_grid_extent]
# The dimension must be between 1 and 3 inclusive
# Cutting corners okay sometimes
#Inputs/Needed Variables
# solution - an object containing points, and times related to path solution
# grid_resx - the resolution of the grid in the x direction
# grid_resy - the resolution of the grid in the y direction
# grid_resz - the resolution of the grid in the z direction
# dim - the dimension of the path to be checked
# timeStep - how much a time should be incremented to walk throught he plynomial path
# aggressParam - a parameter of how aggressive to check a path (0,infty), Closer to 0 means check every point on the path, Closer to infinity => check no points
#Outputs
# occupancy_vec - a vector of the occupancy IDs that the polynomial is characterized by
# outOfBounds - a boolean that is true if the polynomial goes outside of the grid
function occupancyCellChecker(solution::PolySol, grid_resx::Float64, grid_resy::Float64, grid_resz::Float64, dim::Int64, aggressParam::Float64, timeStep::Float64)
    #Check if the dimension is reasonable otherwise print error and exit;
    if(dim < 1 || dim > 3)
        println("Invalid dimension entered")
        return -1;
    end
    
    #Start outOfBounds as false
    outOfBounds = false;

    #Read in the coefficients into a matrix
    coeffMat = [solution.x_coeffs'; solution.y_coeffs'; solution.z_coeffs'];
    #Create a holder for derivatives, delta_t's, and pts when we get there in the for loop
    derivMat = zeros(dim);
    delta_t = zeros(dim);
    pts = zeros(3); #Three is used here since if dim is something other than 3 at this point the values are set to zero
    #prevPt = zeros(dim);
    occupancy_vec = zeros(Int64, 0,1);
    

    
    #Initialize/read in the resolution variables if needed
    #grid_resx = 0.05;
    #grid_resy = 0.05;
    #grid_resz = 0.05;
    #Initialize times assuming polynomials are solve at 0 time initial
    t = 0;
    timeFin = solution.times[end] - solution.times[1];
    #Time step will give you the resolution of the time steps in the loops
    #timeStep = timeFin/1000.0; ##############################################May want to make a variable
    #Calculate the distance time to travel in each direction before getting occupancy grid id using 1st order taylor series approximation
    dist_x = grid_resx * aggressParam;
    dist_y = grid_resy * aggressParam;
    dist_z = grid_resz * aggressParam;
    #Put into a matrix
    dist_to_travel = [dist_x; dist_y; dist_z]
    #Loop for 2 or 3 dimensions if dim >3
    for looper = 1:dim
        #Create previous point vector
        pts[looper] = evaluate_poly(coeffMat[looper,:],0,t);
        #Check if in bounds
        if(pts[looper] < 0 || pts[looper] > get_grid_extent())
            outOfBounds = true;
        end
        #Create a variable to change temporarily
        t_new = t;
        counter = 0;
        #Calculate the time at which the change in distance is more than our distance to travel
        while(abs(pts[looper] - evaluate_poly(coeffMat[looper,:],0,t_new)) < dist_to_travel[looper] && counter < 1000)
            #If we haven't gotten higher than what we want to travel increment the time
            t_new += timeStep;
            #println(t_new)
            counter += 1;
        end
        #println("past first while")
        #Set the new delta_t
        delta_t[looper] = abs(t_new - t);
        #evaluate the polynomial first direvative at current time
        #derivMat[looper] = evaluate_poly(coeffMat[looper, :], 1, t);
        #if(derivMat[looper] == 0)
        #    derivMat[looper] = 1;
        #end
        #looper = 1 -> x, 2 -> y, 3 -> z
        #find the required delta_t's needed to go the set distance before checking occupancy
        #delta_t[looper] = abs(dist_to_travel[looper] / derivMat[looper]);
    end

    #Create a counter
    counter1 =1;

    #Check if at the or past the end time and loop if not passed it
    while(t < timeFin ) #&& counter1 < 100)
        #Find the smallest one
        changeDelta, deltaIndex = findmin(delta_t)
        #Move the time by the smallest one
        t += changeDelta;
        #Decrease the delta t's for the others
        for looper = 1:dim
            #Subract the smallest delta t from the other deltas
            delta_t[looper] -= changeDelta;
        end
        #Find the x,y,z of the current time
        for looper = 1:dim
            pts[looper] = evaluate_poly(coeffMat[looper, :], 0, t);
            #Check if in bounds
            if(pts[looper] < 0 || pts[looper] > get_grid_extent())
                outOfBounds = true;
            end
        end
        #Find occupancy ID at current point by adding to a vector
        occupancy_vec = [occupancy_vec; occupancy_get_id(pts[1], pts[2], pts[3])];
        #println(occupancy_vec)
        #println(changeDelta)
        #Evaluate the new delta_t for the dimension
        for looper = 1:dim
            if(delta_t[looper] == 0)
                #Create previous point vector
                #pts[looper] = evaluate_poly(coeffMat[looper,:],0,t);
                #Create a variable to change temporarily
                t_new = t;
                counter = 0;
                #Calculate the time at which the change in distance is more than our distance to travel
                while(abs(pts[looper] - evaluate_poly(coeffMat[looper,:],0,t_new)) < dist_to_travel[looper] && counter < 1000)
                    #If we haven't gotten higher than what we want to travel increment the time
                    t_new += timeStep;
                    counter += 1;
                end
                #Set the new delta_t
                delta_t[looper] = abs(t_new - t);
            end
        end
        #derivMat[deltaIndex] = evaluate_poly(coeffMat[deltaIndex, :], 1, t);
        #Make sure the derivative is not zero
        #if(derivMat[deltaIndex] == 0)
        #    derivMat[deltaIndex] = 1;
        #end
        #delta_t[deltaIndex] = abs(dist_to_travel[deltaIndex]/derivMat[deltaIndex]);
        #Increment counter
        #counter1 += 1;
    end

        
    #Check the final point just in case based on passed dimension
    if(dim == 3)
        occupancy_vec = [occupancy_vec; occupancy_get_id(evaluate_poly(coeffMat[1,:],0,timeFin),
            evaluate_poly(coeffMat[2,:],0,timeFin),evaluate_poly(coeffMat[3,:],0,timeFin))];
    elseif(dim == 2)
        occupancy_vec = [occupancy_vec; occupancy_get_id(evaluate_poly(coeffMat[1,:],0,timeFin),
            evaluate_poly(coeffMat[2,:],0,timeFin),0)];
    else
        occupancy_vec = [occupancy_vec; occupancy_get_id(evaluate_poly(coeffMat[1,:],0,timeFin),0,0)];
    end
    
    #Return the vector of unique occupancy grids
    return unique(occupancy_vec), outOfBounds;
end


(::#3) (generic function with 1 method)

Int64, Any) in module Main at In[1]:12 overwritten at In[3]:34.


In [1]:

# the unique cell IDs that the path goes through
#Assumptions
# An occupancy function exists that is called as follows ID = occupancy_get_id(x,y,z)
# Path starts at zero
# Solution has coefficient vectors of the same length
#Inputs/Needed Variables
# solution - an object containing points, and times related to path solution
# grid_resx - the resolution of the grid in the x direction
# grid_resy - the resolution of the grid in the y direction
# grid_resz - the resolution of the grid in the z direction
# dim - the dimension of the path to be checked
# aggressParam - a parameter of how aggressive to check a path (0,infty), Closer to 0 means check every point on the path, Closer to infinity => check no points
#Outputs
# occupancy_vec - a vector of the occupancy IDs that the polynomial is characterized by
function occupancyCellChecker(solution::PolySol, grid_resx::Float64, grid_resy::Float64, grid_resz::Float64, dim::Int64, aggressParam::Float64)
    #Check if the dimension is reasonable otherwise print error and exit;
    if(dim < 1 || dim > 3)
        println("Invalid dimension entered")
        return -1;
    end

    #Read in the coefficients into a matrix
    coeffMat = [solution.x_coeffs'; solution.y_coeffs'; solution.z_coeffs'];
    #Create a holder for derivatives, delta_t's, and pts when we get there in the for loop
    derivMat = zeros(dim);
    delta_t = zeros(dim);
    pts = zeros(dim);
    prevPt = zeros(dim);
    occupancy_vec = zeros(Int64,0,1);
        #Create a error value for numeric solver

    error = 0.001
    
    #Initialize/read in the resolution variables if needed
    #grid_resx = 0.05;
    #grid_resy = 0.05;
    #grid_resz = 0.05;
    #Initialize times assuming polynomials are solve at 0 time initial
    t = 0;
    timeFin = solution.times[end] - solution.times[1];
    #Calculate the distance time to travel in each direction before getting occupancy grid id using 1st order taylor series approximation
    dist_x = grid_resx * aggressParam;
    dist_y = grid_resy * aggressParam;
    dist_z = grid_resz * aggressParam;
    #Put into a matrix
    dist_to_travel = [dist_x; dist_y; dist_z]
    #Loop for 2 or 3 dimensions if dim >3
    for looper = 1:dim
        #Create previous point vector
        prevPt[looper] = evaluate_poly(coeffMat[looper,:],0,t);
        #calculate an f(t) to compare with t
        t_new = t;
        f_t = prevPt[looper] + dist_to_travel[looper] + 
            evaluate_poly(coeffMat[looper,:],0,t_new) - coeffMat[looper,1]*t_new;        
        #Calculate the needed delta_t within error
        counter = 0;
        while(abs(t_new - f_t) > error && counter < 1000)
            t_new = f_t;
            f_t = -prevPt[looper] + dist_to_travel[looper] + 
                evaluate_poly(coeffMat[looper,:],0,t_new) - coeffMat[looper,1]*t_new;  
            counter += 1;
        end
        println(t_new)
        #Set the new delta_t
        delta_t[looper] = abs(t_new - t);
        #evaluate the polynomial first direvative at current time
        #derivMat[looper] = evaluate_poly(coeffMat[looper, :], 1, t);
        #if(derivMat[looper] == 0)
        #    derivMat[looper] = 1;
        #end
        #looper = 1 -> x, 2 -> y, 3 -> z
        #find the required delta_t's needed to go the set distance before checking occupancy
        #delta_t[looper] = abs(dist_to_travel[looper] / derivMat[looper]);
    end

    #Create a counter
    counter1 =1;

    #Check if at the or past the end time and while loop if not passed it
    while(t < timeFin && counter1 < 100)
        #Find the smallest one
        changeDelta, deltaIndex = findmin(delta_t)
        #Move the time by the smallest one
        t += changeDelta;
        #Decrease the delta t's for the others
        for looper = 1:dim
            #Subract the smallest delta t from the other deltas
            delta_t[looper] -= changeDelta;
        end
        #Find the x,y,z of the current time
        for looper = 1:dim
            pts[looper] = evaluate_poly(coeffMat[looper, :], 0, t);
        end
        #Find occupancy ID at current point by adding to a vector
        occupancy_vec = [occupancy_vec; occupancy_get_id(pts[1], pts[2], pts[3])];
        #println(occupancy_vec)
        #println(pts)
        #Evaluate the new delta_t for the dimension
        for looper = 1:dim
            if(delta_t[looper] == 0)
                #Create previous point vector
                prevPt[looper] = evaluate_poly(coeffMat[looper,:],0,t);
                #calculate an f(t) to compare with t
                t_new = t;
                f_t = prevPt[looper] + dist_to_travel[looper] + 
                    evaluate_poly(coeffMat[looper,:],0,t_new) - coeffMat[looper,1]*t_new;        

                #Calculate the needed delta_t within error
                counter = 0;
                while(abs(t_new - f_t)> error && counter < 1000)
                    t_new = f_t;
                    f_t = prevPt[looper] + dist_to_travel[looper] + 
                        evaluate_poly(coeffMat[looper,:],0,t_new) - coeffMat[looper,1]*t_new;  
                    counter += 1;
                end
            end
        end
        #derivMat[deltaIndex] = evaluate_poly(coeffMat[deltaIndex, :], 1, t);
        #Make sure the derivative is not zero
        #if(derivMat[deltaIndex] == 0)
        #    derivMat[deltaIndex] = 1;
        #end
        #delta_t[deltaIndex] = abs(dist_to_travel[deltaIndex]/derivMat[deltaIndex]);
        #Increment counter
        counter1 += 1;
    end


    #Return the vector of unique occupancy grids
    return unique(occupancy_vec);
end




LoadError: LoadError: UndefVarError: PolySol not defined
while loading In[1], in expression starting on line 16